In [ ]:
import pyDOE
import pandas as pd
import numpy as np
import Utils 
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.optimize import Bounds
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from collections import namedtuple
from matplotlib import cm
import cma
from scipy.optimize import minimize
from scipy.optimize import Bounds
import sys

## Helper Functions

In [3]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' Original Function for Optimization '''
def Sphere_5D(X):
    X = np.array(X)
    return np.sum(X**2)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=5, samples=n_obs, criterion='m')
    X1,X2,X3,X4,X5 = lhd[:,0],lhd[:,1],lhd[:,2],lhd[:,3],lhd[:,4]
    return X1,X2,X3,X4,X5


'''Robust Regularization based on minimax Principle'''
def robust_regularization(X):
    x1,x2,x3,x4,x5 = X
    eps = np.linspace(-1.0,1.0,1000)
    result = np.zeros(eps.shape[0])
    for i in range(eps.shape[0]):
        result[i]= Sphere_5D(np.array([x1+eps[i],x2+eps[i],x3+eps[i],x4+eps[i],x5+eps[i]]))
    return np.max(result[i])

''' Robustness based on a composite function of Mean and STd '''
def composite_robustness(X):
    w=1
    x1,x2,x3,x4,x5 = X
    np.random.seed(0)
    eps = np.random.normal(loc=0.0,scale=0.166,size=10000)
    sample_points = np.zeros(eps.shape[0])
    for i in range(eps.shape[0]):
        sample_points[i]= Sphere_5D(np.array([x1+eps[i],x2+eps[i],x3+eps[i],x4+eps[i],x5+eps[i]]))
    sample_mean = np.mean(sample_points)
    variance = np.square(sample_points-sample_mean)
    std = np.sqrt(np.mean(variance))
    std = np.sqrt(np.mean(variance))
    return sample_mean + w * std

''' Generate Training Data using LHD along side the Output of the Robust System'''
def generate_training_data(n_obs):
    X1,X2,X3,X4,X5 = DOE(n_obs)
    X1 = linearscaletransform(X1,range_out=(-5,5))
    X2 = linearscaletransform(X2,range_out=(-5,5))
    X3 = linearscaletransform(X3,range_out=(-5,5))
    X4 = linearscaletransform(X4,range_out=(-5,5))
    X5 = linearscaletransform(X5,range_out=(-5,5))
    f_evaluation = np.zeros(X1.shape[0])
    for i in range(X1.shape[0]):
        f_evaluation [i] = Sphere_5D(np.array([X1[i],X2[i],X3[i],X4[i],X5[i]]))
    f_original = np.zeros(X1.shape[0])
    minimax_original = np.zeros(X1.shape[0])
    composite_original = np.zeros(X1.shape[0])
    for i in range(X1.shape[0]):
        f_original[i] = Sphere_5D(np.array([X1[i],X2[i],X3[i],X4[i],X5[i]]))
        minimax_original[i] = robust_regularization(np.array([X1[i],X2[i],X3[i],X4[i],X5[i]]))
        composite_original[i] = composite_robustness(np.array([X1[i],X2[i],X3[i],X4[i],X5[i]]))
    train = pd.DataFrame()
    train['X1'] = pd.Series(X1)
    train['X2'] = pd.Series(X2)
    train['X3'] = pd.Series(X3)
    train['X4'] = pd.Series(X4)
    train['X5'] = pd.Series(X5)
    train['Y = F(X1,X2)'] = pd.Series(f_original)
    train['Y = robust_regularization(X1,X2)'] = pd.Series(minimax_original)
    train['Y = composite(X1,X2)'] = pd.Series(composite_original)
    train.to_csv('Training_Data_Sets\\'+str(n_obs)+'Samples.csv')
    return train

''' Generate Test Data using LHD along side the Output of the Robust System'''
def generate_test_data(n_obs):
    test = pd.DataFrame()
    X = pyDOE.lhs(n=5, samples=n_obs, criterion='m')
    for i in range(X.shape[1]):
        X[:,i] = linearscaletransform(X[:,i],range_out=(-5,5))
        test['X'+str(i+1)] = pd.Series(X[:,i])
    true_minimax = np.zeros(X.shape[0])
    true_composite = np.zeros(X.shape[0])
    for i in range(X.shape[0]):
        true_minimax[i] = robust_regularization(X[i,:])
        true_composite[i] = composite_robustness(X[i,:])
    test ['True_Minimax'] = pd.Series(true_minimax)
    test ['True_Composite'] = pd.Series(true_composite)
    test.to_csv('Test_Data_Sets\\'+str(n_obs)+'Samples.csv')
    return test

In [4]:
dim = 5
multiple = [5,10,15,20,25,30,35,40,45,50]
for i in range(len(multiple)):
    train = generate_training_data(dim * multiple[i])
test = generate_test_data(dim * 75)